<a href="https://colab.research.google.com/github/csch7/CSCI-4170/blob/main/Homework-05/NLP_and_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn

In [2]:
def softmax(scores):
  return torch.exp(scores) / torch.sum(torch.exp(scores), dim = 1)

def scaled_dot_product_attention(queries, keys, values):
  scores = (queries @ keys.T) / torch.sqrt(keys.shape[-1])
  s = softmax(scores)
  return s @ values


# class DotProductAttention(nn.Layer):
#   def __init__(self):
#     super().__init__()

#   def call(self, queries, keys, values):


In [3]:
class Encoder(nn.Module):
  def __init__(self, input_dim, embed_dim, hidden_dim):
    super().__init__()
    self.embed = nn.Embedding(input_dim, embed_dim)
    self.lstm = nn.LSTM(embed_dim, hidden_dim, bidirectional=True)
    self.fc = nn.Linear(2*hidden_dim, hidden_dim)
    self.tanh = nn.Tanh()

  def forward(self, x):
    em = self.embed(x)
    lstm_out, (hidden, _) = self.lstm(em)
    return lstm_out, self.tanh(self.fc(torch.cat((hidden[0], hidden[1]), dim=1)))

class Decoder(nn.Module):
  def __init__(self, embed_dim, hidden_dim, output_dim):
    super().__init()
    self.embed = nn.Embedding(output_dim, embed_dim)
    self.lstm = nn.LSTM(hidden_dim + embed_dim, hidden_dim)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, targets, hidden, encoder_out):
    em = self.embed(targets)
    attn = scaled_dot_product_attention(hidden[-1], encoder_out, encoder_out)
    lstm_out, (hidden, _) = self.lstm(torch.cat((em, attn), dim=2))
    return self.fc(lstm_out), hidden

class Seq2SeqAttn(nn.Module):
  def __init__(self, enc, dec):
    super().__init__()
    self.encoder = enc
    self.decoder = dec

  def forward(self, inputs, targets):
    tar_len = targets.shape[0]
    tar_size = targets.shape[1]
    outputs = torch.zeros(tar_len, tar_size)
    enc_out, hidden = self.encoder(inputs)
    for i in range(1, tar_len):
      dec_out, hidden = self.decoder(targets[i], hidden, enc_out)
      outputs[i] = dec_out
    return outputs


In [6]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.7 MB/s eta 0:00:00


In [17]:
from datasets import load_dataset

ds = load_dataset('bentrevett/multi30k')

train_dat = ds['train'][:len(ds['train'])//20]
valid_dat = ds['validation'][:len(ds['validation'])//20]
test_dat = ds['test'][:len(ds['test'])//20]
train_lab = train_dat['en']
train_dat = train_dat['de']
valid_lab = valid_dat['en']
valid_dat = valid_dat['de']
test_lab = test_dat['en']
test_dat = test_dat['de']

In [18]:
import re

def clean_text(text):
    text = str(text).lower() # Ensure no duplicate word embeddings due to capital letters
    test = re.sub(r',;."()!?&', '', text)         # Remove certain special characters (need to be careful not to remove umlauds or eszetts from German)
    text = re.sub(r"\s+", " ", text).strip()      # Remove extra spaces
    return text

def pad_sentences(dat, vocab, max_len):
  res = torch.empty((len(dat), max_len))
  for s in range(len(dat)):
    res[s,0] =


max_len = 50

sentences_en = [s for ds in [train_lab, valid_lab, test_lab] for s in ds]
vocab_en = set([w for s in sentences_en for w in s])
vocab_en = {word: idx+1 for idx, word in enumerate(vocab_en)}
vocab_en['<PAD>'] = 0

sentences_de = [s for ds in [train_dat, valid_dat, test_dat] for s in ds]
vocab_de = set([w for s in sentences_de for w in s])
vocab_de = {word: idx+1 for idx, word in enumerate(vocab_de)}
vocab_de['<PAD>'] = 0


pad_sentences(train_dat, vocab_de, max_len)


Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.


In [5]:

class PositionalEncoding(nn.Module):
  def __init__(self, seq_len, embedding_dim):
    super().__init__()
    self.seq_len = seq_len
    self.embed_dim = embedding_dim

  def forward(self, x):
    embedding = torch.zeros((self.seq_len, self.embed_dim))
    positions = torch.arange(self.seq_len)
    embedding[positions, ::2] = torch.sin(positions*(10000**(2*torch.arange(self.embed_dim)[:self.embed_dim//2]/self.embed_dim)))
    embedding[positions, 1::2] = torch.cos(positions*(10000**(2*torch.arange(self.embed_dim)[:self.embed_dim//2]/self.embed_dim)))
    return embedding


class ScaledDotProductAttention(nn.Module):
  def __init__(self):
    super().__init__()
    self.sm = nn.Softmax(dim=0)

  def forward(self, q, k, v):
    scores = np.matmul(q, k.T) / np.sqrt(np.shape(k)[-1])
    s = softmax(scores)
    return np.matmul(s, v)

class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, qk, qv, dim_model):
    self.nh = num_heads
    self.d_model = dim_model
    self.Wq = nn.Parameter(torch.randn(num_heads, dim_model, qk))
    self.Wk = nn.Parameter(torch.randn(num_heads, dim_model, qk))
    self.Wv = nn.Parameter(torch.randn(num_heads, dim_model, qv))
    self.Wo = nn.Parameter(torch.randn(num_heads*qv, dim_model))
    self.attn = ScaledDotProductAttention()

  def forward(self, Q, K, V):
    heads = self.attn(Q @ self.Wq, K @ self.Wk, V @ self.Wv)
    output = heads[0]
    for i in range(1, self.nh):
      output = torch.cat(output, heads[i])


class FFN(nn.Module):
  def __init__(self, embedding_dim = 64, hidden_dim = 128):
    super().__init__()
    self.w1 = nn.Parameter(torch.randn(embedding_dim, hidden_dim))
    self.b1 = nn.Parameter(torch.randn(hidden_dim))
    self.w2 = nn.Parameter(torch.randn(hidden_dim, embedding_dim))
    self.b2 = nn.Parameter(torch.randn(embedding_dim))

  def forward(self, x):
    return (torch.max(0, (x @ self.w1) + self.b1) @ self.w2) + self.b2


class Encoder(nn.Module):
  def __init__(self, input_len, vocab_size, d_model = 64, hidden_dim = 128, num_heads = 8, num_layers = 2):
    super().__init__()
    self.d_model = d_model
    self.hidden_dim = hidden_dim
    self.embed = nn.Embedding(vocab_size, d_model)
    self.position = PositionalEncoding(input_len, d_model)
    self.ffn = FFN(d_model, hidden_dim)
    self.layernorm = nn.LayerNorm((input_len, d_model))
    self.attn = MultiHeadAttention(num_heads, d_model / num_heads, d_model / num_heads, d_model)
    self.L = num_layers

  def forward(self, inputs):
    em = self.embed(inputs)
    pos_en = self.position(inputs)
    out = em + pos_en
    for l in range(self.L):
      self_attn = self.attn(out, out, out)
      attn_norm = self.layernorm(self_attn + out)
      ffn_out = self.FFN(attn_norm)
      out = self.layernorm(ffn_out + attn_norm)
    return out


class Decoder(nn.Module):
  def __init__(self, output_len, vocab_size, d_model = 64, hidden_dim = 128, num_heads = 8, num_layers = 2):
    super().__init__()
    self.d_model = d_model
    self.hidden_dim = hidden_dim
    self.embed = nn.Embedding(vocab_size, d_model)
    self.position = PositionalEncoding(output_len, d_model)
    self.ffn = FFN(d_model, hidden_dim)
    self.layernorm = nn.LayerNorm((output_len, d_model))
    self.attn = MultiHeadAttention(num_heads, d_model / num_heads, d_model / num_heads, d_model)
    self.masked_attn = MultiHeadAttention(num_heads, d_model / num_heads, d_model / num_heads, d_model) # FIGURE OUT MASK
    self.L = num_layers

  def forward(self, outputs, enc_out):
    em = self.embed(outputs)
    pos_en = self.position(outputs)
    out = em + pos_en
    for l in range(self.L):
      self_attn = self.masked_attn(out, out, out)
      attn_norm = self.layernorm(self_attn + out)
      enc_attn = self.attn(enc_out, enc_out, attn_norm)
      attn_norm = self.layernorm(enc_attn + attn_norm)
      ffn_out = self.FFN(attn_norm)
      out = self.layernorm(ffn_out + attn_norm)
    return out


class Transformer(nn.Module):
  def __init__(self, input_len, output_len, vocab_size, d_model = 64, hidden_dim = 128, num_heads = 8, num_layers = 2):
    super().__init__()
    self.encoder = Encoder(input_len)
    self.decoder = Decoder(output_len)
    self.fc = nn.Linear(d_model, vocab_size)
    self.sm = nn.Softmax(dim=0)

  def forward(self, inputs, outputs):
    enc_out = self.encoder(inputs)
    dec_out = self.decoder(outputs, enc_out)
    return self.sm(self.fc(dec_out))